In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
%load_ext autoreload

In [ ]:
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)

In [ ]:
import os
import numpy as np
import nibabel as nib
import nibabel.orientations as nio
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from skimage import io
from patchify import patchify, unpatchify
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
from numpy import load
import glob
import sys
import json
import time
from PIL import Image, ImageDraw
from pathlib import Path
import os
import shutil
import re
import zipfile

import matplotlib.pyplot as plt
import cv2

import numpy as np
import pickle
from tqdm import tqdm

import io
import base64
from IPython.display import HTML

In [ ]:
# Loss Function and coefficients to be used during training:
def dice_coe(y_true,y_pred, loss_type='jaccard', smooth=1.):

    y_true_f = tf.reshape(y_true,[-1])
    y_pred_f = tf.reshape(y_pred,[-1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f)

    if loss_type == 'jaccard':
        union = tf.reduce_sum(tf.square(y_pred_f)) + tf.reduce_sum(tf.square(y_true_f))

    elif loss_type == 'sorensen':
        union = tf.reduce_sum(y_pred_f) + tf.reduce_sum(y_true_f)

    else:
        raise ValueError("Unknown `loss_type`: %s" % loss_type)

    return (2. * intersection + smooth) / (union + smooth)

In [ ]:
def dice_loss(y_true,y_pred, loss_type='jaccard', smooth=1.):

    y_true_f = tf.cast(tf.reshape(y_true,[-1]),tf.float32)
    y_pred_f =tf.cast(tf.reshape(y_pred,[-1]),tf.float32)

    intersection = tf.reduce_sum(y_true_f * y_pred_f)

    if loss_type == 'jaccard':
        union = tf.reduce_sum(tf.square(y_pred_f)) + tf.reduce_sum(tf.square(y_true_f))

    elif loss_type == 'sorensen':
        union = tf.reduce_sum(y_pred_f) + tf.reduce_sum(y_true_f)

    else:
        raise ValueError("Unknown `loss_type`: %s" % loss_type)

    return (1-(2. * intersection + smooth) / (union + smooth))

In [ ]:
activation = 'sigmoid'
patch_size_x =1120 # should be a multiple of 32 
patch_size_y =1120
n_classes = 1
channels=3

LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)
# optim = tf.keras.optimizers.SGD(LR, momentum=0.9)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, Dropout, concatenate, UpSampling2D, BatchNormalization
import tensorflow as tf

def Unet2D(inputs,num_classes):
    x=inputs
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same',data_format="channels_last")(x)
    conv1= BatchNormalization()(conv1)
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same')(conv1)
    conv1= BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same')(pool1)
    conv2= BatchNormalization()(conv2)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same')(conv2)
    conv2= BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same')(pool2)
    conv3= BatchNormalization()(conv3)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same')(conv3)
    conv3= BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same')(pool3)
    conv4= BatchNormalization()(conv4)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv4)
    conv4= BatchNormalization()(conv4)
    drop4 = Dropout(0.01)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same')(pool4)
    conv5= BatchNormalization()(conv5)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv5)
    conv5= BatchNormalization()(conv5)
    drop5 = Dropout(0.01)(conv5)
    up6 = Conv2D(64, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(drop5))
    up6= BatchNormalization()(up6)
    merge6 = concatenate([drop4,up6],axis=-1)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same')(merge6)
    conv6= BatchNormalization()(conv6)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv6)
    conv6= BatchNormalization()(conv6)

    up7 = Conv2D(32, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6))
    up7= BatchNormalization()(up7)
    merge7 = concatenate([conv3,up7],axis=-1)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same')(merge7)
    conv7= BatchNormalization()(conv7)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same')(conv7)
    conv7= BatchNormalization()(conv7)

    up8 = Conv2D(16, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7))
    up8= BatchNormalization()(up8)
    merge8 = concatenate([conv2,up8],axis=-1)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same')(merge8)
    conv8= BatchNormalization()(conv8)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same')(conv8)
    conv8= BatchNormalization()(conv8)

    up9 = Conv2D(8, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
    up9= BatchNormalization()(up9)
    merge9 = concatenate([conv1,up9],axis=-1)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same')(merge9)
    conv9= BatchNormalization()(conv9)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same')(conv9)
    conv9= BatchNormalization()(conv9)
#     conv10 = Conv3D(1, 1, activation = 'sigmoid')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs=inputs, outputs = conv10)
    #model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
inputs = tf.keras.Input(shape=(patch_size_x,patch_size_y, channels))
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"]) 

# with mirrored_strategy.scope():
Model_2D = Unet2D(inputs,num_classes=n_classes)
Model_2D.compile(optimizer=optim, loss=dice_loss, metrics=[dice_coe])
Model_2D.summary()

In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K
from PIL import Image



def parse_image(img_path, image_size):
    image_rgb = cv2.resize((np.array(Image.open(img_path))).astype('float32'), (1120,1120)) #.jpg
    train_img = image_rgb.copy()
    return train_img

def parse_mask(mask_path, image_size):
    mask=np.array(Image.open(mask_path))
    mask=np.where(mask,1,0)
    nm = cv2.resize((mask.astype('float32')), (1120,1120)) #.jpg
    train_mask = np.expand_dims(nm, axis=2)
    return train_mask


class DataGen(Sequence):
    def __init__(self, image_size, images_path, masks_path, batch_size=8):
        self.image_size = image_size
        self.images_path = images_path
        self.masks_path = masks_path
        self.batch_size = batch_size
        self.on_epoch_end()

    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.images_path):
            self.batch_size = len(self.images_path) - index*self.batch_size
        images_path = self.images_path[index*self.batch_size : (index+1)*self.batch_size]
        masks_path = self.masks_path[index*self.batch_size : (index+1)*self.batch_size]

        images_batch = []
        masks_batch = []

        for i in range(len(images_path)):
            # Read image and mask
            image = parse_image(images_path[i], self.image_size)
            mask = parse_mask(masks_path[i], self.image_size)

            images_batch.append(image)
#             mask=np.where(mask>1,1,mask)
            masks_batch.append(mask)

        return np.array(images_batch), np.array(masks_batch)

    def on_epoch_end(self):
        pass

    def __len__(self):
        return int(np.ceil(len(self.images_path)/float(self.batch_size)))
    

In [ ]:
train_data_paths = sorted(glob.glob("/home/centos/rabeea/maski/cate2/images/*.jpg"))
train_masks_paths = sorted(glob.glob("/home/centos/rabeea/maski/cate2/masks/*.bmp"))



valid_data_paths = sorted(glob.glob("/home/centos/rabeea/maski/cate2/images/*.jpg"))
valid_masks_paths = sorted(glob.glob("/home/centos/rabeea/maski/cate2/masks/*.bmp"))

In [ ]:
image_size = 1120
batch_size = 1

epochs = 100

train_steps = len(train_data_paths)//batch_size
valid_steps = len(valid_data_paths)//batch_size

## Generator
train_gen = DataGen(image_size, train_data_paths,train_masks_paths, batch_size=batch_size)
valid_gen = DataGen(image_size,valid_data_paths, valid_masks_paths, batch_size=batch_size)

In [ ]:
train_gen.__getitem__(20)[0].shape

In [ ]:
train_gen.__getitem__(20)[1].shape

In [ ]:
history=Model_2D.fit(train_gen,
                            validation_data=valid_gen,epochs=500,shuffle=True
                              )

In [ ]:
Model_2D.save('/home/centos/rabeea/maski/2Dunet_teeth_cate2.h5')

In [ ]:
from tensorflow.keras.models import load_model
Model_2D = load_model('/home/centos/rabeea/maski/2Dunet_teeth_cate3.h5', compile=False)
# my_model=Model_3D

In [ ]:
#plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['dice_coe']
val_acc = history.history['val_dice_coe']

plt.plot(epochs, acc, 'y', label='Training Dice coefficient')
plt.plot(epochs, val_acc, 'r', label='Validation Dice Coefficient')
plt.title('Training and validation Dice coefficient')
plt.xlabel('Epochs')
plt.ylabel('Dice Coefficient')
plt.legend()
plt.show()

In [ ]:
ppath= [n.split('/')[-1].split('.')[0] for n in sorted(glob.glob("/home/centos/rabeea/maski/cate3/instance_masks/*.bmp"))]
data_paths = []
masks_paths = []
for p in (ppath):
    data_paths.append(os.path.join('/home/centos/rabeea/maski/cate3/images/',(p+'.jpg'))) 
    masks_paths.append(os.path.join('/home/centos/rabeea/maski/cate3/masks/',(p+'.bmp'))) 
for k in range(len(data_paths)):
    test_data_paths=[]
    test_masks_paths=[]
    test_data_paths.append(data_paths[k])
    test_masks_paths.append(masks_paths[k])
    image_size = 1120
    batch_size = 1
    test_steps = len(test_data_paths)//batch_size
    test_gen = DataGen(image_size, test_data_paths,test_masks_paths, batch_size=batch_size)
    y_pred=Model_2D.predict(test_gen.__getitem__(0)[0])
    y_pred_argmax=np.where(y_pred>0.8,1,0)
    pred=y_pred_argmax[0,:,:,0].astype('float32')
    pred= cv2.resize(pred, (1991, 1127))
    pred=np.where(pred>0,255,0).astype(np.uint8)
#     plt.imshow(pred)
#     plt.show()
#     print((pred))
    im=Image.fromarray(pred)
    im.save(os.path.join('/home/centos/rabeea/maski/cate3/pred_masks/',(data_paths[k].split('/')[-1].split('.')[0]+'.bmp')))

In [ ]:
test_data_paths = sorted(glob.glob("/home/centos/rabeea/maski/cate2/images/*.jpg"))[0:1]
test_masks_paths = sorted(glob.glob("/home/centos/rabeea/maski/cate2/masks/*.bmp"))[0:1]
image_size = 1120
batch_size = 1

epochs = 100

test_steps = len(test_data_paths)//batch_size

## Generator
test_gen = DataGen(image_size, test_data_paths,test_masks_paths, batch_size=batch_size)

In [ ]:
print(test_data_paths)

In [ ]:
np.shape(test_gen.__getitem__(0)[0])

In [ ]:
y_pred=Model_2D.predict(train_gen.__getitem__(19)[0])
y_pred_argmax=np.where(y_pred>0.8,1,0)
true_msk=train_gen.__getitem__(19)[1]

In [ ]:
np.shape(y_pred_argmax)

In [ ]:
# gt= np.zeros([128,128],dtype=int)
# pred= np.zeros([128,128],dtype=int)
# for i in range (33):
#     gt= np.where(true_msk[0,:,:,i]==1,i,gt)
#     pred= np.where(y_pred_argmax[0,:,:,i]==1,i,pred)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(true_msk[0,:,:,0],'gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(y_pred_argmax[0,:,:,0],'gray')
plt.show()

In [ ]:
pred=y_pred_argmax[0,:,:,0].astype('float32')
pred= cv2.resize(pred, (1991, 1127))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(pred)
plt.show()

In [ ]:
gt=true_msk[0,:,:,0].astype('float32')
gt= cv2.resize(gt, (1991, 1127) )

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(gt)
plt.show()

In [ ]:
original= (train_gen.__getitem__(19)[0][0]/255).astype('float32')
original= cv2.resize(original, (1991, 1127), interpolation= cv2.INTER_CUBIC )

In [ ]:
plt.figure(figsize=(10,10))
# plt.imshow((train_gen.__getitem__(5)[0][0]/255))
plt.imshow(original)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
image=original.copy()
pred=np.where(pred>0,True,False)
image[pred]= (0,0,255)
plt.imshow(image)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
image=original.copy()
gt=np.where(gt>0,True,False)
image[gt]= (0,0,255)
plt.imshow(image)
plt.show()

In [ ]:
new=Image.blend(original, image, .7)
plt.figure(figsize=(10,10))
plt.imshow(new)
plt.show()

In [ ]:
np.unique(pred)

In [ ]:
np.unique(train_gen.__getitem__(5)[0][0]/255)

In [ ]:
image[mask] = (0, 0, 255)

In [ ]:
np.shape(train_gen.__getitem__(2)[0][0])

In [ ]:
gt=gt.astype('float32')
gt= cv2.resize(gt, (1991, 1127))
original= cv2.resize(train_gen.__getitem__(2)[0][0].astype('float32'), (1991,1127))
plt.figure(figsize=(10,10))
# plt.imshow(original,alpha=0.5)
plt.imshow(gt)
plt.show()

In [ ]:
np.unique(original)

In [ ]:
np.unique(gt)